<span style="color:red"><b>  생각해보기  </b></span>  <br>
* data에 outlier에 대해서도 해야하니 quantile regression 이 적합?? <br>
    -> probability 처럼 나타낼 수도 있음 <br>
    
    
* 바람 쌘 날이 잘 반영되지 않음 <br>
    -> 어떤 새로운 변수를 만들어야 하나??<br>
    
    
* TMA Capacity 정의 <br>
    TMA와 Aiport의 Capacity에 차이라고 한다면 holding으로 인한 것 뿐인 것 같음 <br>
    -> 이 부분만 airport capacity에 포함을 시키면 되지않을까? <br>
    
  
* 이 ML이 prediction이라고 주는 값은,  Demand가 ~고, 날씨가 ~일 떄 몇대가 들어올 것 같다는 값을 줌 <br>
    -> 이 항공기 숫자를 robust하게 예측하게 하고(확률적으로) reservior 개념을 적용해서 scheduling problem으로 발전(holding은 어떻게 넣을래?)  <br>
    
    
* 
<span style="color:red"> *** </span>
<b> reservoir를 하려면 AAR, ADR을 STAR, SID에 들어오는 수로 해야겠는데??? (특히 Arrival) </b> <br>
    그래야지 TMA용량 정의를 holding하는 항공기 수를 포함하여 적용가능<br>
    departure는 거의 공항과 같게 될 것 같음(departure는 홀딩을 거의 공항에서 하므로) , 아니면 departure로 hand off되는 고도에 도착하는 걸로  <br><br>
    
* Max capacity에 영향을 주는 것 <br>
관제사 워크로드 <br>
기상 <br>
항공기 성능(어떤 어프로치 가능한지 등) <br>
<br>
<br>

***    
<br>
예를들어, reservoir에서 15분까지 airborne holding까지 허용한다고 하자 <br>
지금 방법으로하면 Demand만 한대씩 늘려가면서, 몇 대가 들어올 지 예측해서 값을 주는 거지 <br>
(즉, A대가 그떄 도착하도록 이렇게저렇게 GDP를 조정해서 보낸다 하면, 그 중 몇대가 B대가 그시간대에 올지 예측해서 값으로 줌) <br>
-> 이렇게 한대씩 늘렸을 때 예측값으로 준 rate과 불확실성을 대변하는 무엇(별표*)을 가지고 최대 15분 holding이라는 기준을 맞추도록 stochastic scheduling을 해서 최종적으로 적절하게 보내도 되는 A값을 찾음 (여러 곳에서 오는 신호를 순서대로 처리하라고 보내는 것이라고 할 수도 있음)  <br>
-> A값을 그 상황에서의 Capacity 로 정의 <br>
<br>
별표(*) <br>
몇 대가 올지 예측한 거를 확률로?<br>
근데 어디서 올지, 언제 올지를 모른단 말이지 -> 이것도 확률로??<br>

***

# Data Preparation

In [29]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna.integration.lightgbm as lgb          ####
from lightgbm import LGBMRegressor, plot_metric    #### 여기서 lgb로 안가져옴
from pathlib import Path
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, max_error, mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error

In [30]:
# options
pd.set_option('max_columns',100)
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')
seed = 1

In [31]:
# Data dirctory
data_dir = Path('../eda/')
data_file = data_dir / 'Data.csv'

In [32]:
# Data
# 0:AAR / 1:EAD / 2:ADR / 3:EDD는 고정  , 나머지는 순서 상관 없음
Data = pd.read_csv(data_file, index_col=0)
Data_6 = pd.read_csv(data_dir / 'Data_6.csv' , index_col=0)
Data_12 = pd.read_csv(data_dir / 'Data_12.csv', index_col=0)
Data_18 = pd.read_csv(data_dir / 'Data_18.csv', index_col=0)
Data_24 = pd.read_csv(data_dir / 'Data_24.csv', index_col=0)

Data

,AAR,EAD,ADR,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_400,WD_500,WD_700,WD_850,WD_925,WD_1000,WS_400,WS_500,WS_700,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,TMP,TD,PS,PA,RN,HM,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6,WDIR_t12,WSPD_t12,WG_t12,VIS_t12,WC_t12,CLA_1LYR_t12,BASE_1LYR_t12,CLA_2LYR_t12,BASE_2LYR_t12,CLA_3LYR_t12,BASE_3LYR_t12,WDIR_t18,WSPD_t18,WG_t18,VIS_t18,WC_t18,CLA_1LYR_t18,BASE_1LYR_t18,CLA_2LYR_t18,BASE_2LYR_t18,CLA_3LYR_t18,BASE_3LYR_t18,WDIR_t24,WSPD_t24,WG_t24,VIS_t24,WC_t24,CLA_1LYR_t24,BASE_1LYR_t24,CLA_2LYR_t24,BASE_2LYR_t24,CLA_3LYR_t24,BASE_3LYR_t24
0,4.0,3.0,7.0,9.0,2019,1,1,0,2,1.0,1.0,0.0,0.0,0.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,1,6,0.0,1000,1,-53,-102,10359,10350,0.0,68,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
1,2.0,3.0,6.0,7.0,2019,1,1,1,2,1.0,1.0,4.0,7.0,0.0,2.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,31,5,0.0,1000,1,-57,-105,10358,10349,0.0,69,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
2,0.0,2.0,3.0,3.0,2019,1,1,2,2,1.0,1.0,2.0,6.0,1.0,1.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,33,4,0.0,1000,1,-60,-105,10359,10350,0.0,70,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
3,6.0,1.0,5.0,2.0,2019,1,1,3,2,1.0,1.0,0.0,3.0,2.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,32,4,0.0,1000,1,-63,-112,10363,10354,0.0,68,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
4,20.0,15.0,1.0,2.0,2019,1,1,4,2,1.0,1.0,6.0,5.0,0.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,7,1,0.0,1000,1,-66,-118,10357,10348,0.0,66,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,26.0,28.0,33.0,31.0,2019,12,31,19,2,1.0,1.0,25.0,25.0,4.0,10.0,305.0,305.0,315.0,320.0,37.0,22.0,22.0,11.0,0.0,0.0,0.0,0.0,32,8,0.0,1000,1,-61,-137,10358,10349,0.0,54,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
8756,39.0,29.0,32.0,40.0,2019,12,31,20,2,1.0,1.0,26.0,33.0,2.0,0.0,305.0,305.0,315.0,320.0,37.0,22.0,22.0,11.0,0.0,0.0,0.0,0.0,33,9,0.0,1000,1,-54,-127,10357,10348,0.0,56,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,320.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
8757,23.0,28.0,32.0,23.0,2019,12,31,21,2,1.0,1.0,39.

In [27]:
A = Data[Data['EDD'] == 0]

A[A['ADR'] == 30]

,AAR,EAD,ADR,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_400,WD_500,WD_700,WD_850,WD_925,WD_1000,WS_400,WS_500,WS_700,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,TMP,TD,PS,PA,RN,HM,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6,WDIR_t12,WSPD_t12,WG_t12,VIS_t12,WC_t12,CLA_1LYR_t12,BASE_1LYR_t12,CLA_2LYR_t12,BASE_2LYR_t12,CLA_3LYR_t12,BASE_3LYR_t12,WDIR_t18,WSPD_t18,WG_t18,VIS_t18,WC_t18,CLA_1LYR_t18,BASE_1LYR_t18,CLA_2LYR_t18,BASE_2LYR_t18,CLA_3LYR_t18,BASE_3LYR_t18,WDIR_t24,WSPD_t24,WG_t24,VIS_t24,WC_t24,CLA_1LYR_t24,BASE_1LYR_t24,CLA_2LYR_t24,BASE_2LYR_t24,CLA_3LYR_t24,BASE_3LYR_t24
4943,11.0,7.0,30.0,0.0,2019,7,25,23,4,2.0,2.0,28.0,24.0,0.0,0.0,245.0,230.0,235.0,230.0,210.0,0.0,12.0,18.0,28.0,24.0,29.0,0.0,21,7,0.0,600,5,244,221,10045,10037,0.0,87,8,5.0,1.0,6.0,20.0,8.0,80.0,0.0,400.0,1000.0,190.0,10.0,0.0,1400.0,5,4,10.0,7,25.0,8,80.0,190.0,10.0,0.0,1400.0,5,4,10.0,7,25.0,8,80.0,190.0,10.0,0.0,2300.0,5,4,10.0,7,25.0,8,80.0,190.0,10.0,0.0,4200.0,5,4,10.0,7,25.0,8,80.0


Validation <br><br>

sklearn.model_selection
* train_test_split
* KFold
* StratifiedKFold

<br>
<br>
lightGBM <br>
* lgb.cv    # 쓰려면 lightgbm을 lgb로 import

***
***
***

# LightGBM fitting

## Data Split

In [ ]:
# 예측할 시간에 맞는 Data로 넣기
# 0-6 : Data_6 / 6-12 : Data_12 / 12-18 : Data_18 / 18-24 : Data_24
Data_raw = Data_6
Data_m = Data_6
Data_m = Data_m.drop('AAR', axis=1)
Data_m = Data_m.drop('ADR', axis=1)


# Arrival
y_a = Data_raw.AAR.to_numpy()
X_a = Data_m.to_numpy()
#X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.1, random_state = seed)
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.1, random_state = seed)
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a, y_train_a, test_size=0.11, random_state = 13) 


# Departure
y_d = Data_raw.ADR.to_numpy()
X_d = Data_m.to_numpy()
#X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size = 0.1, random_state = seed)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size = 0.1, random_state = seed)
X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X_train_d, y_train_d, test_size=0.11, random_state = 13) 

# val은 hyperparameter 검증에 사용
# 0.11 x 0.9 = 0.099

print('Training Data shape : ', Data_m.shape)
Data_m.head()

***
## Hyperparameter Optimization

In [ ]:
# Hyperparameters
params = {'boosting_type' : 'gbdt',            # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
          'metric': 'mse',
          'num_leaves' : 127,                  ## Maximum tree leaves for base learners (31)
          'max_depth' : - 1,                   # Maximum tree depth for base learners, <=0 means no limit (-1)
          'learning_rate' : 0.001,             ## Boosting learning rate (0.1)
          'n_estimators' : 10000000,           # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
          'subsample_for_bin' : 200000,        # Number of samples for constructing bins (200000)
          'objective' : 'regression',          # learning task and the corresponding learning objective (None)
          'class_weight' : None,               # * Use this parameter only for multi-class classification task
          'min_split_gain' : 0.0,              # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
          'min_child_weight' : 0.001,          # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
          'min_child_samples' : 1,             # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
          'subsample' : 0.8,                   ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
          'subsample_freq' : 1,                # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
          'colsample_bytree' : 0.8,            ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
          'reg_alpha' : 0.0,                   # L1 regularization term on weights (0)
          'reg_lambda' : 0.0,                  # L2 regularization term on weights (0)
          'random_state' : seed,               # Random number seed (None)
          'n_jobs' : - 1,                      # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
          'silent' : True,                     # Whether to print messages while running boosting (True)
          'importance_type' : 'split'}         # ‘split’: result contains numbers of times the feature is used in a model
                                               # ‘gain’ : result contains total gains of splits which use the feature

# 최적 hyperparameter 찾기!!

In [ ]:
# Hyperparameters - arrival
params_a = {'boosting_type' : 'gbdt',                   # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                          ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                          # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                    ## Boosting learning rate (0.1)
            'n_estimators' : 1000000,                   # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,               # Number of samples for constructing bins (200000)
            'objective' : 'regression',                 # learning task and the corresponding learning objective (None)
            'class_weight' : None,                      # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                     # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,                 # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                    # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.8,           ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,   #3                  # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.6839999999999999,    ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 1.5913347234314888e-05,       # L1 regularization term on weights (0)
            'reg_lambda' : 0.8083105161094011,           # L2 regularization term on weights (0)
            'random_state' : seed,                      # Random number seed (None)
            'n_jobs' : - 1,                             # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                            # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}                # ‘split’: result contains numbers of times the feature is used in a model
                                                        # ‘gain’ : result contains total gains of splits which use the feature

In [ ]:
# Optuna 간단 버전 - arrival

"""
y = Data_raw.AAR.to_numpy()
X = Data_m.to_numpy()

X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_a = lgb.Dataset(X_train_a, label = y_train_a)
dval_a = lgb.Dataset(X_val_a, label = y_val_a)

model = lgb.train(params, dtrain_a,
                  valid_sets=[dtrain_a, dval_a], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_a, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_a, prediction)        # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯



params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_a = params
"""

In [ ]:
# Hyperparameters - departure
params_d = {'boosting_type' : 'gbdt',                 # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                        ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                        # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                  ## Boosting learning rate (0.1)
            'n_estimators' : 1000000,                 # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,             # Number of samples for constructing bins (200000)
            'objective' : 'regression',               # learning task and the corresponding learning objective (None)
            'class_weight' : None,                    # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                   # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,               # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                  # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.9220698151647799,         ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,    #3               # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.8480000000000001,  ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 1.2560334090582146e-07,     # L1 regularization term on weights (0)
            'reg_lambda' : 0.003315287591858434,      # L2 regularization term on weights (0)
            'random_state' : seed,                    # Random number seed (None)
            'n_jobs' : - 1,                           # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                          # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}              # ‘split’: result contains numbers of times the feature is used in a model
                                                      # ‘gain’ : result contains total gains of splits which use the feature

In [ ]:
# Optuna 간단 버전 - departure

"""
y = Data_raw.ADR.to_numpy()
X = Data_m.to_numpy()

X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_d = lgb.Dataset(X_train_d, label = y_train_d)
dval_d = lgb.Dataset(X_val_d, label = y_val_d)

model = lgb.train(params, dtrain_d,
                  valid_sets=[dtrain_d, dval_d], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_d, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_d, prediction)         # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯

params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_d = params
"""

***
## Arrival

In [ ]:
# Model fitting
reg_arrival = LGBMRegressor(**params_a)

reg_arrival.fit(X_train_a, y_train_a,
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_val_a, y_val_a)],        # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier
                early_stopping_rounds = 10,             # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
                verbose = False)            

***
##  Departure

In [ ]:
# Model fitting
reg_departure = LGBMRegressor(**params_d)

reg_departure.fit(X_train_d, y_train_d,
                  eval_set=[(X_val_d, y_val_d)],
                  eval_metric='l2',
                  early_stopping_rounds = 10, 
                  verbose = False)     # loss fuction이 n번 이상 좋아지지 않으면 멈춰라

***

## Cross validation

for loop안에서 predict하고 평균을 내야 함 <br>
-> demand를 하나씩 증가시키리면, 바로 CV를 사용하기는 어려울 듯

In [ ]:
"""
n_fold = 10

cv = KFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_val_a = np.zeros(X_a.shape[0])
predict_a = np.zeros(X_a.shape[0])
p_val_d = np.zeros(X_d.shape[0])
predict_d = np.zeros(X_d.shape[0])

############### 위에서 X_val_a등으로 바꾼 거 없애고 -> 아래 X_a, y_a, X_d, y_d를 X_train_a 등으로 바꾸기 #######################

for i, (i_trn, i_val) in enumerate(cv.split(X_a, y_a), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_arrival = LGBMRegressor(**params)    #############params_a
    reg_arrival.fit(X_a[i_trn], y_a[i_trn],
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_a[i_val], y_a[i_val])],  # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_a[i_val] = reg_arrival.predict(X_a[i_val])
    predict_a += reg_arrival.predict(X_a) / n_fold      ##### 이 자리에 원래는 test data가 들어가면 됨
                                                        # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
    
for i, (i_trn, i_val) in enumerate(cv.split(X_d, y_d), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_departure = LGBMRegressor(**params)    #############params_d
    reg_departure.fit(X_d[i_trn], y_d[i_trn],
                      sample_weight = None,                   # Weights of training data
                      init_score = None,                      # Weights of training data
                      eval_set = [(X_d[i_val], y_d[i_val])],  # pairs to use as validation sets
                      eval_sample_weight = None,              # Weights of eval data
                      eval_init_score = None,                 # Init score of eval data.   
                      eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                      early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_d[i_val] = reg_departure.predict(X_d[i_val])
    predict_d += reg_departure.predict(X_d) / n_fold          ##### 이 자리에 원래는 test data가 들어가면 됨
                                                              # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
"""

In [ ]:
"""
print(f'Arrival RMSE : {np.sqrt(mean_squared_error(y_a, predict_a)):.4f}')
print(f'Arrival Training R^2 : {r2_score(y_a, predict_a) * 100:.4f}')
print(f'Departure RMSE : {np.sqrt(mean_squared_error(y_d, predict_d)):.4f}')
print(f'Departure Training R^2 : {r2_score(y_d, predict_d) * 100:.4f}')

np.savetxt('p_val_a.csv', p_val_a, fmt='%.6f', delimiter=',')
np.savetxt('p_val_d.csv', p_val_d, fmt='%.6f', delimiter=',')
np.savetxt('predict_a.csv', predict_a, fmt='%.6f', delimiter=',')
np.savetxt('predict_d.csv', predict_d, fmt='%.6f', delimiter=',')
"""

이렇게 각각의 model의(e.g. lightGBM, XGBOOST, Randomforest ...) Cross Validation을 예측한 결과와, Test data에 대해 예측한 결과를 파일로 저장 <br>
-> 다음 stage에서는 CV결과들을 input data로 사용, 기존의 label은 그대로 사용, test data의 예측값을 다음 stage에서 test data의 input으로 사용 <br>
-> 계속 쌓아갈 수 있음 (=Stacking, 보통 stage 1,2정도면 이후 효과는 미비)


stacking, ensemble 보다 feature engineering, hyperparameter tuning의 성능 향상이 훨씬 커서 앞의 방법은 굳이 사용하지 않아도 괜찮음

***
***
***

# Evaluation

<sklearn.metrics> <br>

* regression <br>
mean_squared_error, mean_absolute_error, r2_score <br><br>

* classification <br>
log_loss, roc_auc_score, accuracy_score, confusion_matrix <br>

In [ ]:
def Test(Data_raw, ops='arrival', start=0, end = 10):        # ops : arrival , departure
    if ops == 'arrival':
        Data_raw = Data_raw.drop('ADR', axis=1)
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EAD' : Data_raw['EAD'][start:end],
                                    'Actual AAR' : Data_raw['AAR'][start:end], 
                                    'Predicted AAR' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]), 
                                    'Difference' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]) - Data_raw['AAR'][start:end]})
    elif ops == 'departure':
        Data_raw = Data_raw.drop('AAR', axis=1)        
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EDD' : Data_raw['EDD'][start:end],
                                    'Actual ADR' : Data_raw['ADR'][start:end], 
                                    'Predicted ADR' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]), 
                                    'Difference' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]) - Data_raw['ADR'][start:end]})
        
    return test_result

***
## Arrival

In [ ]:
# Predict & Evaluation

y_hat_a = reg_arrival.predict(X_a)
print(f'Total RMSE : {np.sqrt(mean_squared_error(y_a, y_hat_a)):.4f}')

print(f'Training R^2 : {r2_score(y_train_a, reg_arrival.predict(X_train_a)) :.4f}')
print(f'Test R^2 : {r2_score(y_test_a, reg_arrival.predict(X_test_a)) :.4f}')

print(f'Training RMSE : {np.sqrt(mean_squared_error(y_train_a, reg_arrival.predict(X_train_a))):.4f}')
print(f'Test RMSE : {np.sqrt(mean_squared_error(y_test_a, reg_arrival.predict(X_test_a))):.4f}')
#print(f'explained variance score : {explained_variance_score(y_a, y_hat_a) :.4f}')
#print(f'max error : {max_error(y_a, y_hat_a) :.4f}')
#print(f'mean absolute error : {mean_absolute_error(y_a, y_hat_a) :.4f}')
#print(f'mean absolute percentage error : {mean_absolute_percentage_error(y_a, y_hat_a)}')
#print(f'mean squared log error : {mean_squared_log_error(y_a, y_hat_a) :.4f}')
#print(f'median absolute error : {median_absolute_error(y_a, y_hat_a) :.4f}')


In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_m.columns, 'importance': reg_arrival.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_arrival, figsize=(10,10))

In [ ]:
# AAR - pred
pred_test_a = Data_raw.drop('ADR', axis=1)
pred_test_a['pred_a'] = reg_arrival.predict(pred_test_a.drop('AAR', axis=1))
sns.pairplot(data=pred_test_a, vars=['AAR','pred_a'], size=10, plot_kws={'alpha': .5}, kind ='scatter' )

In [ ]:
# Test
Test_all = Test(Data_raw, ops='arrival', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

In [ ]:
Test(Data_raw, ops='arrival', start = 0, end = 8760)

-------------------
## Departure

In [ ]:
# Predict & Evaluation
print(f'Total RMSE : {np.sqrt(mean_squared_error(y_d, reg_departure.predict(X_d))):.4f}')

print(f'Training R^2 : {r2_score(y_train_d, reg_departure.predict(X_train_d)) :.4f}')
print(f'Test R^2 : {r2_score(y_test_d, reg_departure.predict(X_test_d)) :.4f}')

print(f'Training RMSE : {np.sqrt(mean_squared_error(y_train_d, reg_departure.predict(X_train_d))):.4f}')
print(f'Test RMSE : {np.sqrt(mean_squared_error(y_test_d, reg_departure.predict(X_test_d))):.4f}')

In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_m.columns, 'importance': reg_departure.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_departure, figsize=(10,10))

In [ ]:
# ADR - pred
pred_test_d = Data_raw.drop('AAR', axis=1)
pred_test_d['Prediction_departure'] = reg_departure.predict(pred_test_d.drop('ADR', axis=1))
sns.pairplot(data=pred_test_d, vars=['ADR','Prediction_departure'], size=10, plot_kws={'alpha': .5})

In [ ]:
#############################################################################################################
#############################################################################################################
#############################################################################################################

fig = plt.figure(figsize=(20, 10))

ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)


sns.scatterplot(data = pred_test_d, x = "ADR", y = "Prediction_departure" , ax = ax1)
sns.distplot(pred_test_d["ADR"], label = 'ADR', kde = False, ax = ax2)
sns.distplot(pred_test_d["Prediction_departure"], label = 'Prediction', kde = False, ax = ax2)

ax1.set(xlabel='Actual Departure Rate', ylabel='Departure Prediction')
ax2.set(xlabel='Departures per hour', ylabel='Count')
#ax1.set_title('aaa')
#ax2.set_title('bbb')

plt.legend()
plt.show()

#############################################################################################################
#############################################################################################################
#############################################################################################################

In [ ]:
# Test
Test_all = Test(Data_raw, ops='departure', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

***
***
***

# Maximum Capacity

In [ ]:
"""

# 예전 버전 : 원래 demand부터 시작 -> extra demand를 더함

def max_capacity(Data_raw, Data_m, example):   
    
    # extra
    extra_demand = 50
    
    # arrival
    data_a = Data_m.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    for i in range(extra_demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i-1
    XXX_a = XX_a[1:extra_demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[extra_demand-1,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:extra_demand+1])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw['AAR'][example:example+1])
    ead = int(Data_raw['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_m[example:example+1]))
    
    # departure
    data_d = Data_m.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    for i in range(extra_demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,2] = XX_d[i,2] + i-1
    XXX_d = XX_d[1:extra_demand+1]
    XXXX_d = np.arange(XXX_d[0,2], XXX_d[extra_demand-1,2]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:extra_demand+1])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw['ADR'][example:example+1])
    edd = int(Data_raw['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_m[example:example+1])) 

    # Total
    result_ = pd.DataFrame({'EAD' : XXXX_a ,'Predicted AAR' : YYYY_a, 'EDD' : XXXX_d ,'Predicted ADR' : YYYY_d, 
                            'Demand' : XXXX_a + YYYY_a, 'Capacity' : YYYY_a + YYYY_d})
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')
    plt.plot(XXXX_a[:-15]+XXXX_d[:-15], YYYY_a[:-15] + YYYY_d[:-15], label = 'Total')   # 안 예뻐서 뒤에 15개 자름
    plt.legend(prop={'size': 20})
    ax = plt.subplot()
    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.3f}\n
               Predicted AAR : {prediction_a:.3f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.3f}\n
               Predicted ADR : {prediction_d:.3f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.3f}\n
               Predicted Rate : {prediction_a + prediction_d:.3f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    plt.show()    
    return result_
    
"""

In [ ]:
# *** training data에는 AAR, ADR 둘다 없어야 함
# 0부터 시작
# max capacity 선 추가
# 원래 demand에 대한 predict 점 추가

def max_capacity(Data_raw, Data_m, example):    # 50까지 늘림
    
    # extra
    demand = 80
    

    # arrival
    data_a = Data_m.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    
    original_demand_a = int(data_a[0][0])
    for i in range(0,demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i - original_demand_a
    
    XX_a[0] = XX_a[1]   
    XX_a[0,0] = 0   
    XXX_a = XX_a[0:demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[demand,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:demand+1])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw.drop('ADR', axis=1)['AAR'][example:example+1])
    ead = int(Data_raw.drop('ADR', axis=1)['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_m[example:example+1]))
    
    
    
    # departure
    data_d = Data_m.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    
    original_demand_d = int(data_d[0][1])
    for i in range(0,demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,1] = XX_d[i,1] + i - original_demand_d
        
    XX_d[0] = XX_d[1]           
    XX_d[0,1] = 0
    XXX_d = XX_d[0:demand+1]
    XXXX_d = np.arange(XXX_d[0,1], XXX_d[demand,1]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:demand+1])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw.drop('AAR', axis=1)['ADR'][example:example+1])
    edd = int(Data_raw.drop('AAR', axis=1)['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_m[example:example+1])) 

    
    
    # capacity - arrival과 departure을 수요의 비율로 늘렸을 때
    max_cap = int(max(Data_raw['AAR'] + Data_raw['ADR']))
    max_capacity = np.zeros(demand+1)
    for i in range(0,demand*2+1,2):
        if round(i*data_a[0,0]/(data_a[0,0]+data_d[0,1])) >=75:
            capa_arr = YYYY_a[75]
        elif round(i*data_d[0,1]/(data_a[0,0]+data_d[0,1])) >=75:
            capa_dep = YYYY_d[75]
        else:
            capa_arr = YYYY_a[round(i*data_a[0,0]/(data_a[0,0]+data_d[0,1]))]
            capa_dep = YYYY_d[round(i*data_d[0,1]/(data_a[0,0]+data_d[0,1]))]
        max_capacity[int(i/2)] = capa_arr + capa_dep
    
    
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    ax = plt.subplot()

    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')    # Arrival
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')    # Departure
    plt.plot(XXXX_a[:-5] + XXXX_d[:-5], max_capacity[:-5], label = 'Total')    # Capacity - 안 예뻐서 뒤에 55개 자름
    plt.plot(XXXX_a[:-5]+XXXX_d[:-5], [max_cap]*(demand-4), label = f'Empirical Maximum = {max_cap}')    # 데이터 상 max capacity
    
    plt.plot(data_a[0,0], prediction_a,'ob', markersize = 10)    # 원래 arrival 예측값
    plt.plot(data_d[0,1], prediction_d,'or', markersize = 10)    # 원래 departure 예측값
    plt.plot(data_a[0,0]+data_d[0,1], prediction_a + prediction_d,'yo', markersize = 15)    # 원래 capacity 예측값
    plt.legend(prop={'size': 20}, loc = 'upper left')

    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.1f}\n
               Predicted AAR : {prediction_a:.1f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.1f}\n
               Predicted ADR : {prediction_d:.1f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.1f}\n
               Predicted Rate : {prediction_a + prediction_d:.1f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    
    plt.show()   
    
    return Data_raw[example:example+1]   #result_

In [ ]:
# 원하는 시간의 Max Capacity 그래프

max_capacity(Data_raw, Data_m, 154)     # 숫자에 원하는 Data index(0-8760) 넣기

< Notable Samples> 

* pd.DataFrame(Data_raw).idxmax()             # 각 column별 max값의 index

AAR 최소 : 2 <br>
AAR 최대 : 1481 <br>
EAD 최대 : 641 <br>
ADR 최소 : 28 <br>
ADR 최대 : 538 <br>
EDD 최대 : 273 <br>
Total Rate(AAR+ADR) 최대(77) : 5703 <br>
Total Demand(EAD+EDD) 최대(70) : 4814 <br><br>

Arrival_reaminder 최대 : 5992 <br>
Departure_remainder 최대 : 8242 <br><br>

WSPD 최대 : 5990 <br>
W_GST 최대 : 5990 <br>
시정 최소 : 1755 <br>
Ceiling 최소 : 321 <br>
RVR 최소 1448 <br><br>

TAF WSPD 최대 : 5988 <br>
TAF W_GST 최대 : 5988 <br>
TAF 시정 최소 : 319 <br>
TAF Ceiling 최소 : 10?? <br>

In [ ]:
# Data : 불러온 파일
# Data_temp : 일단 필요 없어 보이는 column을 버린 것
# data_taf : dictinoary 안에 Data_6 - Data_24 전부 넣은 것
# Data_6, Data_12 .. : 해당 시간 전의 TAF만 있고, 나머지 시간의 TAF는 버린 것
# Data_raw : Data_#과 동일
# Data_m : Data_# 에서 AAR, ADR 뺀 것
# Data_a : Data_m 로 바꿈
# Data_d : Data_m 로 바꿈

***
***
***

# 추가 실험

##  왜 AAR에서 중간이 비는가?

In [ ]:
a = reg_arrival.predict(X_a)
np.rint(a)        # 반올림하고 정수로 만듦
pd.DataFrame(np.rint(a)).value_counts().sort_index()

In [ ]:
# AAR수 별로 정렬
Data['AAR'].value_counts().sort_index()

In [ ]:
# 12에서 16으로 예측한 날들 표시
pred = pd.DataFrame({'pred' :reg_arrival.predict(Data_m)})
pred_low = pred[(pred['pred'] < 16) & (pred['pred'] >= 12)].index
Data.loc[pred_low]

***
***
***

# 모델 저장

In [ ]:
import joblib

lgbr_a = "../evaluate/lgbr_a.pkl"
lgbr_d = "../evaluate/lgbr_d.pkl"
joblib.dump(reg_arrival, lgbr_a)
joblib.dump(reg_departure, lgbr_d)